In [ ]:
%%bash
git add nlp100_80.ipynb

In [ ]:
%%bash
git commit -a -m "snapshot"

****0 コーパスの整形 ****


文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ

In [1]:
import bz2

In [2]:
import codecs

In [3]:
fname_input = "nlp100data/enwiki-20150112-400-r100-10576.txt.bz2"

In [4]:
fname_output = "nlp100data/corpus80.txt"

- bz2.open の引数"rt"の"t"はtextを表す。省略するとbyte型になってしまう。
- chunk.strip()は引数を指定しないとwhitespaceを除去

In [ ]:
with bz2.open(fname_input  , 'rt') as rf , codecs.open(fname_output , "w" , "utf-8") as wf:
    for i , l in enumerate(rf):
        if i > -1:
            tokens = []
            for chunk in l.split(' '):
                token = chunk.strip().strip('.,!?;:()[]\'"')
                if len(token) > 0:
                    tokens.append(token)
                    
            processed = " ".join(tokens) + "\n"
            wf.write(processed)

                


In [ ]:
 " ".join(tokens) + "\n"

#### 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

https://gist.github.com/kalinchernev/486393efcca01623b18d

In [5]:
import re

In [6]:
fname_output81 = "nlp100data/output81.txt"

In [ ]:
%%bash
cd nlp100data

wget https://gist.githubusercontent.com/kalinchernev/486393efcca01623b18d/raw/daa24c9fea66afb7d68f8d69f0c4b8eeb9406e83/countries

In [ ]:
with codecs.open("nlp100data/countries"
                 , "r" , "utf-8") as rf:
    country_names = [l.strip('\n') for l in rf]

    

In [ ]:
country_names

In [ ]:
len(country_names)

In [ ]:
%%bash
git commit -a -m "doing 81"

In [ ]:
compound_countries = dict()
for c in country_names:
    c_split = c.split(" ")
    n = len(c_split)
    if n > 1:
        compound_countries[c] = "_".join(c_split)
        

In [ ]:
with codecs.open(fname_output , "r" , "utf-8") as rf , codecs.open(fname_output81 , "w" , "utf-8") as wf:
    for i, l in enumerate(rf):
        for k ,v in compound_countries.items():
            l = l.replace(k , v)
        #print(l)
        wf.write(l)
        #if i > 10:
         #   break

In [ ]:
%%bash
git commit -a -m "finished 81"

##### sandbox

In [ ]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

In [ ]:
for k  , v in compound_countries.items():
    print(k , v)

#### 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は{1,2,3,4,5}の範囲でランダムに決める．





In [5]:
import pickle

In [6]:
import random

In [7]:
fname_list82 = "nlp100data/list82.pkl"

In [ ]:
random.randint(1,5)

In [ ]:
list82 = list()
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        v = l.strip().split(' ')
        
        
        for j , w in enumerate(v):
            d = random.randint(1 , 5)
            vcontext = [v[k] for k in set(range(j-d , j + d + 1)) & set(range(len(v))) - set([j]) ]
            if len(w) * len(vcontext) >0:
                list82.append(w + "\t" + " ".join(vcontext))
        if i > 99999999999:
            break

In [ ]:
with open(fname_list82 , "wb") as wf:
    pickle.dump(list82 , wf)

In [ ]:
len(tmp)


In [ ]:
len(list82)

In [ ]:
[print(l) for i , l in enumerate(list82) if i < 10] 

##### sandbox

In [ ]:
len(vcontext)

In [ ]:
w.strip?

In [ ]:
%%bash
git commit -a -m "done 82"

In [ ]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

#### 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,∗)$: 単語$t$の出現回数
- $f(∗,c)$: 文脈語$c$の出現回数
- $N$:単語と文脈語のペアの総出現回数

In [11]:
fname_tc_counter83 = "nlp100data/tc_counter83.pkl"
fname_t_counter83 = "nlp100data/t_counter83.pkl"
fname_c_counter83 = "nlp100data/c_counter83.pkl"

In [12]:
import pickle

In [13]:
from collections import Counter

In [ ]:
with open(fname_list82 , "rb") as rf:
    list82 = pickle.load(rf)

In [ ]:
tc_counter = Counter()
t_counter = Counter()
c_counter = Counter()
for i , l in enumerate(list82):
    tc_counter.update([l])
    v = l.split("\t")
    t_counter.update([v[0]])
    c_counter.update([v[1]])
    if i > 999999999:
        break
    

In [ ]:
v[0]

In [ ]:
c_counter.most_common()

In [ ]:
with open(fname_tc_counter83 , "wb") as wf:
    pickle.dump(tc_counter , wf)

In [ ]:
with open(fname_t_counter83 , "wb") as wf:
    pickle.dump(t_counter , wf)

In [ ]:
with open(fname_c_counter83 , "wb") as wf:
    pickle.dump(c_counter , wf)

In [ ]:
%%bash
cd nlp100data

ls -l   *83.pkl

In [ ]:
%%bash
git commit -a -m "doing 84"

##### counterの使い方

###### unique elementの数

In [ ]:

len(t_counter)

In [ ]:
len(tc_counter)

In [ ]:
len(c_counter)


###### key , countの抽出

In [ ]:
tmp = t_counter.items()

In [ ]:
list(tmp)

In [ ]:
list(t_counter.keys())[0:10]

In [ ]:
list(t_counter.values())[0:10]

###### count合計

In [ ]:
sum(t_counter.values())

In [ ]:
sum(tc_counter.values())

##### sandbox

In [ ]:
import ngram

In [ ]:
t_counter['Personal']

#### 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

- $f(t,c)≥10$ならば，$X_{tc}= {\rm PPMI}(t,c) == \max\{\log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
- $f(t,c)<10$ならば，$X_{tc}=0$


ここで，${\rm PPMI}(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．

なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は0になるので，非0の要素だけを書き出せばよい．

In [18]:
fname_matrix84 = "nlp100data/matrix84"

In [19]:
fname_dict_t = "nlp100data/dict_t84.pkl"
fname_dict_c = "nlp100data/dict_c84.pkl"

In [20]:
import math

In [21]:
from scipy import io

In [22]:
from scipy.sparse import lil_matrix

In [ ]:
with open(fname_tc_counter83 , "rb") as rf:
    tc_counter = pickle.load(rf)

In [ ]:
with open(fname_t_counter83 , "rb") as rf:
    t_counter = pickle.load(rf)

In [ ]:
with open(fname_c_counter83 , "rb") as rf:
    c_counter = pickle.load(rf)

In [ ]:
N = sum(tc_counter.values())

In [ ]:
t_list = list(t_counter.keys())

In [ ]:
c_list = list(c_counter.keys())

In [ ]:
size_t = len(t_list)

In [ ]:
size_c = len(c_list)

In [ ]:
matrix84 = lil_matrix((size_t , size_c))

In [ ]:
from collections import OrderedDict

In [ ]:
dict_t = OrderedDict()
for i , t in enumerate(t_counter.keys()):
    dict_t[t] = i

In [ ]:
dict_c = OrderedDict()
for i , c in enumerate(c_counter.keys()):
    dict_c[c] = i

In [ ]:
for k , f_tc in tc_counter.items():
    if f_tc >= 10:
        #print(".")
        tokens = k.split("\t")
        t = tokens[0]
        c = tokens[1]
        i = dict_t[t]
        j = dict_c[c]
        ppmi = max(math.log(N * f_tc) / (t_counter[t] * c_counter[c]), 0)
        matrix84[i , j] = ppmi

In [ ]:
matrix84.shape

In [ ]:
io.savemat(fname_matrix84 , {'matrix84' : matrix84})

In [ ]:
with open(fname_dict_t , "wb") as wf:
    pickle.dump(dict_t , wf)

In [ ]:
with open(fname_dict_c , "wb") as wf:
    pickle.dump(dict_c , wf)

##### sparse matrix types

[解説](https://datascience.stackexchange.com/questions/31352/understanding-scipy-sparse-matrix-types)

```
lilとかは行列をセットするときに使い、CSRなどに変換してから各種演算を行え
```
とある

[解説２](http://d.hatena.ne.jp/billest/20090906/1252269157)
saveにはio.savematを使う旨書いてある。

##### sandbox

In [ ]:
dict_t = dict()
for i , t in enumerate(t_counter.keys()):
    dict_t[t] = i

In [ ]:
dict_c = dict()
for i , c in enumerate(c_counter.keys()):
    dict_c[c] = i

In [ ]:
for k , f_tc in tc_counter.items():
    if f_tc >= 10:
        tokens = k.split("\t")
        t = tokens[0]
        c = tokens[1]
        i = t_list.index(t)
        j = c_list.index(c)
        ppmi = max(math.log(N * f_tc) / (t_counter[t] * c_counter[c]), 0)
        matrix84[i , j] = ppmi

In [ ]:
%%bash
git commit -a -m "84 done"

#### 主成分分析による次元圧縮

84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

[いろいろ手法がある](http://jakevdp.github.io/blog/2012/12/19/sparse-svds-in-python/)

In [28]:
import sys

In [48]:
fname_matrix85 = "nlp100data/matrix85"

In [38]:
matrix84 = io.loadmat(fname_matrix84)['matrix84']

In [30]:
matrix84.shape

(388634, 10636393)

In [36]:
print("matrix 84 consumes {} bytes of memory".format(matrix84.data.nbytes))

matrix 84 consumes 58848 bytes of memory


In [33]:
matrix84.asfptype()

<388634x10636393 sparse matrix of type '<class 'numpy.float64'>'
	with 7356 stored elements in Compressed Sparse Column format>

In [39]:
matrix84.dtype

dtype('<f8')

##### scipy.sparse.linalgによる方法

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html

- 良さそうにみえるがメモリエラーになりがち.
- ５０次元くらいならいけるので暫定的にこちらの値を採用。

In [40]:
from scipy.sparse.linalg import svds, eigs

In [41]:
type(matrix84)

scipy.sparse.csc.csc_matrix

In [42]:
matrix84.asfptype()

<388634x10636393 sparse matrix of type '<class 'numpy.float64'>'
	with 7356 stored elements in Compressed Sparse Column format>

In [43]:
u,s,vt = svds(matrix84,50)

In [130]:
io.savemat(fname_matrix85 ,{'matrix85' : u})

In [128]:
u.shape

(388634, 50)

In [129]:
u.transpose().dot(u)

array([[  1.00000000e+00,  -1.00956889e-17,   3.15710491e-16, ...,
          1.61908587e-16,   9.76486627e-17,  -3.03064304e-16],
       [ -1.00956889e-17,   1.00000000e+00,   1.38481332e-17, ...,
         -1.13110582e-16,   4.45668590e-16,  -9.27052931e-18],
       [  3.15710491e-16,   1.38481332e-17,   1.00000000e+00, ...,
          4.62995376e-16,   5.76448687e-17,   5.35944503e-19],
       ..., 
       [  1.61908587e-16,  -1.13110582e-16,   4.62995376e-16, ...,
          1.00000000e+00,   9.27352766e-16,   1.62093891e-16],
       [  9.76486627e-17,   4.45668590e-16,   5.76448687e-17, ...,
          9.27352766e-16,   1.00000000e+00,  -5.16692653e-16],
       [ -3.03064304e-16,  -9.27052931e-18,   5.35944503e-19, ...,
          1.62093891e-16,  -5.16692653e-16,   1.00000000e+00]])

In [45]:
s.shape

(50,)

In [46]:
vt.shape

(50, 10636393)

##### truncatedSVD : よく言及されている方法

アウトプットは列数が圧縮された行列。これはPCAではなくてLSAの模様

In [104]:
from sklearn.decomposition import TruncatedSVD

In [106]:
clf = TruncatedSVD(5)

In [107]:
matrix85 = clf.fit_transform(matrix84)


In [115]:
np.linalg.norm(matrix85[1,:])

3.9106053761339993e-21

In [112]:
np.sum(np.power(matrix85[:,1],2))

0.011957574581204789

###### 出力はsparse.linalg.svdsのuと本質的に同じ

２次元に落としたとき、以下のようにcosine similarityが２（逆向き）、１（直行）,0(一致)になることからわかる：

In [57]:
from scipy.spatial.distance import cosine

In [65]:
u,s,vt = svds(matrix84,2)

In [66]:
clf = TruncatedSVD(2)

In [67]:
u2 = clf.fit_transform(matrix84)

In [71]:
cosine(u[:,0] , u2[:,0])

0.99999999999640865

In [72]:
cosine(u[:,0] , u2[:,1])

6.191758217255483e-11

In [73]:
cosine(u[:,1] , u2[:,0])

1.9999999999999969

In [74]:
cosine(u[:,1] , u2[:,1])

0.99999999999683742

##### scipy.linalg.svdによる方法

そもそもsparse matrixに対応してない

In [26]:
from scipy.linalg import svd

In [27]:
u , s ,vt = svd(matrix84, 50)

ValueError: Sparse matrices are not supported by this function. Perhaps one of the scipy.sparse.linalg functions would work instead.

##### 普通のsvdの挙動チェック

https://stackoverflow.com/questions/24913232/using-numpy-np-linalg-svd-for-singular-value-decomposition

In [116]:

X = np.random.normal(size=[20,18])
P, D, Q = np.linalg.svd(X, full_matrices=False)

$P^{\rm T} P$が単位行列。つまり$P$の各列が直行している。

In [125]:
P.shape

(20, 6)

In [126]:
P.transpose().dot(P)

array([[  1.00000000e+00,   8.17858622e-17,  -1.15540569e-16,
          6.04263603e-17,   6.01869419e-17,  -6.97376595e-17],
       [  8.17858622e-17,   1.00000000e+00,  -1.10107572e-16,
         -6.90529333e-17,  -2.02994316e-16,  -5.16226845e-17],
       [ -1.15540569e-16,  -1.10107572e-16,   1.00000000e+00,
         -8.32667268e-17,  -8.32667268e-17,  -5.55111512e-17],
       [  6.04263603e-17,  -6.90529333e-17,  -8.32667268e-17,
          1.00000000e+00,  -9.95072313e-17,   3.86843335e-16],
       [  6.01869419e-17,  -2.02994316e-16,  -8.32667268e-17,
         -9.95072313e-17,   1.00000000e+00,  -2.49800181e-16],
       [ -6.97376595e-17,  -5.16226845e-17,  -5.55111512e-17,
          3.86843335e-16,  -2.49800181e-16,   1.00000000e+00]])

$P$の各列のL2 normもちゃんと１に正規化されている。

In [127]:
np.linalg.norm(P , axis = 0)

array([ 1.,  1.,  1.,  1.,  1.,  1.])

In [124]:
P,d,q = svds(X)

##### sandbox

In [31]:
type(matrix85)

numpy.ndarray

In [32]:
matrix85.shape

NameError: name 'matrix85' is not defined

In [50]:
%%bash

git commit -a -m "85 done"

[master 38153e5] 85 done
 1 file changed, 176 insertions(+), 105 deletions(-)


##### randomized PCA

In [34]:
from sklearn.decomposition import PCA

In [41]:
pca = PCA(svd_solver='randomized' , n_components = 2)

In [43]:
pca.fit(matrix84.toarray())

MemoryError: 

In [38]:
from sklearn.decomposition import RandomizedPCA

In [39]:
pca = RandomizedPCA(2)

/home/toshinao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


In [40]:
pca.fit(matrix84)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [34]:
%%bash 
free

              total        used        free      shared  buff/cache   available
Mem:       16357616     5745120     8061308      215576     2551188     9985600
Swap:       4193276     3621564      571712


#### 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ

In [51]:
with open(fname_dict_t , "rb") as rf:
    dict_t = pickle.load(rf)

In [55]:
i = dict_t['United_States']

In [114]:
matrix85 = io.loadmat(fname_matrix85)['matrix85']

In [56]:
united_states_vec = matrix85[i , :]

In [57]:
%%bash 
git commit -a -m "86 done"

[master 5be22a9] 86 done
 1 file changed, 49 insertions(+), 4 deletions(-)


####  単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．


In [65]:
from scipy.spatial.distance import cosine

In [58]:
with open(fname_dict_t , "rb") as rf:
    dict_t = pickle.load(rf)

In [59]:
i = dict_t['United_States']

In [62]:
j = dict_t['U.S']

In [131]:
v0 = matrix85[i,:]

In [132]:
v1 = matrix85[j,:]

In [68]:
print("cosine similarity is : {}".format(cosine(v0,v1)))

cosine similarity is : 3.986363572572316e-09


cosine関数の出力は２（逆向き）、１（直行）,0(一致)

In [70]:
%%bash
git commit -a -m "done 87"

[master cfcfbd6] done 87
 1 file changed, 122 insertions(+), 3 deletions(-)


#### 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [73]:
import numpy as np

In [71]:
with open(fname_dict_t , "rb") as rf:
    dict_t = pickle.load(rf)

In [ ]:
matrix85 = io.loadmat(fname_matrix85)['matrix85']

np.linalg.normは二乗norm(sqrt後）を返す.

ここでは「行ごとの」L2 normを計算しておく

In [133]:
norm85 = np.linalg.norm(matrix85 , axis=1)

ちゃんと列ごとのL2 normが１になってることのチェック:

In [154]:
k = dict_t['England']

In [138]:
norm85[k]

4.1047264331981662e-06

In [155]:
v2 = matrix85[k , :]

In [164]:
v2.transpose().shape

(50,)

In [165]:
inner88 = matrix85.dot(v2)

In [166]:
cos88 = inner88 / (norm85 * norm85[k])

In [167]:
cos88.shape

(388634,)

In [173]:
np.argsort(cos88)[::-1][0:10]

array([315334,  66049, 192727, 369151,  97409, 125380, 274635, 348491,
       157253, 367594])

In [169]:
cos88[k]

1.0

In [170]:
k

157253

In [180]:
[(list(dict_t.items())[i] , cos88[i]) for i in np.argsort(cos88)[::-1][0:10]]

[(('Because', 315334), 1.0000000000000002),
 (('arms', 66049), 1.0000000000000002),
 (('Season', 192727), 1.0000000000000002),
 (('Construction', 369151), 1.0000000000000002),
 (('History', 97409), 1.0000000000000002),
 (('Part', 125380), 1.0000000000000002),
 (('Much', 274635), 1.0000000000000002),
 (('Coat', 348491), 1.0),
 (('England', 157253), 1.0),
 (('List', 367594), 1.0)]

In [181]:
list(dict_t.items())[k][0]

'England'

In [136]:
np.linalg.norm(matrix85 , axis = 0)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [171]:
np.max(cos88)

1.0000000000000002

In [182]:
##bash
git commit -a -m "doing 88"

SyntaxError: invalid syntax (<ipython-input-182-b8011ceda251>, line 2)